# WebSocket Orderbook Client
Connects to mock WS and reconstructs mid.

In [ ]:
import asyncio, json, websockets, matplotlib.pyplot as plt
async def run():
    uri='ws://localhost:8000/ws/orderbook/binance/BTCUSDT?depth=10&interval_ms=500'
    xs=[]
    async with websockets.connect(uri) as ws:
        for _ in range(30):
            msg=json.loads(await ws.recv()); mid=(msg['bids'][0][0]+msg['asks'][0][0])/2
            xs.append(mid)
    return xs
try:
    arr=asyncio.get_event_loop().run_until_complete(run())
    plt.plot(arr); plt.title('WS mid'); plt.show()
except Exception as e:
    print('Start mock_apis first:', e)
